In [ ]:
import os
import openai
import pandas as pd
import pandasql as psql
from fpdf import FPDF

# -------------------------------
# 1. Securely Load OpenAI API Key
openai.api_key = 'API here'

if not openai.api_key:
    raise ValueError("OpenAI API key not found. Please set the 'OPENAI_API_KEY' environment variable.")
# -------------------------------

# -------------------------------
# 2. Load CSV Files into DataFrames
# -------------------------------

def load_data():
    try:
        t_zacks_fc = pd.read_csv("t_zacks_fc.csv")
        t_zacks_fr = pd.read_csv("t_zacks_fr.csv")
        t_zacks_mktv = pd.read_csv("t_zacks_mktv.csv")
        t_zacks_shrs = pd.read_csv("t_zacks_shrs.csv")
        t_zacks_sectors = pd.read_csv("t_zacks_sectors.csv")
        print("CSV files loaded successfully.")
        return {
            "t_zacks_fc": t_zacks_fc,
            "t_zacks_fr": t_zacks_fr,
            "t_zacks_mktv": t_zacks_mktv,
            "t_zacks_shrs": t_zacks_shrs,
            "t_zacks_sectors": t_zacks_sectors,
        }
    except Exception as e:
        print(f"Error loading CSV files: {e}")
        return {}

# -------------------------------
# 3. Execute SQL Query on DataFrames
# -------------------------------

def run_query(query, dataframes):
    try:
        # Optional: Log the available tables
        print("Available tables:", list(dataframes.keys()))
        
        # Optional: Display first few rows of each DataFrame for verification
        for name, df in dataframes.items():
            print(f"\nDataFrame: {name}")
            print(df.head())
        
        result = psql.sqldf(query, dataframes)
        print("SQL query executed successfully.")
        return result
    except Exception as e:
        print(f"Error executing SQL query: {e}")
        return pd.DataFrame()

# -------------------------------
# 4. Generate Analysis with ChatGPT
# -------------------------------

def generate_analysis_from_chatgpt(dataframe):
    if dataframe.empty:
        return "No data available for analysis."

    table_md = dataframe.to_markdown(index=False)
    prompt = f"""
I have the following data table from a query on company financials and market data:

{table_md}

Please analyze this data in the style of an equity analyst, highlighting trends, comparing companies, and noting any interesting insights regarding this data.
"""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.7,
        )
        analysis = response['choices'][0]['message']['content'].strip()
        print("Analysis generated successfully.")
        return analysis
    except openai.error.OpenAIError as e:
        return f"An error occurred while generating analysis: {e}"

# -------------------------------
# 5. Define PDF Generation Class
# -------------------------------

class PDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 16)
        self.cell(0, 10, "Equity Analyst Report", align="C", ln=True)
        self.ln(10)

    def chapter_title(self, title):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, title, 0, 1, "L")
        self.ln(4)

    def chapter_body(self, body):
        self.set_font("Arial", "", 12)
        for line in body.split('\n'):
            self.multi_cell(0, 10, line)
        self.ln()

    def table(self, data):
        if data.empty:
            self.set_font("Arial", "I", 12)
            self.cell(0, 10, "No data available to display.", 0, 1, 'C')
            self.ln()
            return

        self.set_font("Arial", "B", 10)
        col_widths = self.calculate_col_widths(data)

        # Add table headers
        for header in data.columns:
            self.cell(col_widths[header], 10, header, 1, 0, 'C')
        self.ln()

        # Add table rows
        self.set_font("Arial", "", 10)
        for _, row in data.iterrows():
            for header in data.columns:
                cell_text = str(row[header]) if pd.notnull(row[header]) else ""
                self.cell(col_widths[header], 10, cell_text, 1, 0, 'C')
            self.ln()

        self.ln()

    def calculate_col_widths(self, data):
        col_widths = {}
        for col in data.columns:
            max_length = data[col].astype(str).map(len).max()
            header_length = len(col)
            col_width = max(max_length, header_length) * 2
            col_width = min(max(col_width, 30), 60)
            col_widths[col] = col_width
        return col_widths

# -------------------------------
# 6. Generate PDF Report
# -------------------------------

def generate_pdf_report(analysis_text, data_table, filename="equity_analyst_report.pdf"):
    pdf = PDF()
    pdf.add_page()

    # Add Overview
    pdf.chapter_title("Overview of Selected Companies in the Technology Sector")
    overview_text = (
        "This report provides an analysis of the largest technology companies based on market capitalization, "
        "including data on revenue, net income, and market capitalization."
    )
    pdf.chapter_body(overview_text)

    # Add Analysis
    pdf.chapter_title("Analysis")
    pdf.chapter_body(analysis_text)

    # Add Data Table
    pdf.chapter_title("Company Financial Data")
    pdf.table(data_table)

    # Save PDF
    try:
        pdf.output(filename)
        print(f"Report generated and saved as {filename}")
    except Exception as e:
        print(f"Error saving PDF: {e}")

# -------------------------------
# 7. Interactive Chat with ChatGPT
# -------------------------------

def communicate_with_chatgpt(prompt):
    """
    Sends a single prompt to ChatGPT and retrieves the response.

    Args:
        prompt (str): The user's input prompt.

    Returns:
        str: The response from ChatGPT.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.7,
            n=1,
            stop=None
        )

        # Extract the assistant's reply
        assistant_reply = response.choices[0].message.content.strip()
        return assistant_reply

    except openai.error.OpenAIError as e:
        return f"An error occurred: {e}"

def interactive_chat():
    """
    Initiates an interactive conversation with ChatGPT, maintaining context.
    """
    print("Start chatting with ChatGPT (type 'exit' or 'quit' to stop):")
    conversation = [
        {"role": "system", "content": "You are a helpful assistant."}
    ]

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Ending the chat. Goodbye!")
            break

        # Append the user's message to the conversation
        conversation.append({"role": "user", "content": user_input})

        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation,
                max_tokens=150,
                temperature=0.7,
                n=1,
                stop=None
            )

            assistant_reply = response.choices[0].message.content.strip()
            print("ChatGPT:", assistant_reply)

            # Append the assistant's reply to the conversation
            conversation.append({"role": "assistant", "content": assistant_reply})

        except openai.error.OpenAIError as e:
            print(f"An error occurred: {e}")

# -------------------------------
# 8. Main Function to Integrate All Functionalities
# -------------------------------

def main():
    # Load data
    dataframes = load_data()
    if not dataframes:
        print("Failed to load data. Exiting.")
        return

    # Define SQL query (adjust table names as per loaded data)
    sql_query = """
SELECT *
FROM t_zacks_mktv
WHERE mkt_val > 1000000;
    """

    # Execute query
    query_result = run_query(sql_query, dataframes)
    if query_result.empty:
        print("No data returned from the SQL query.")

    # Display the result (for debugging)
    print("Query Result:")
    print(query_result)

    # Generate analysis using ChatGPT
    analysis_text = generate_analysis_from_chatgpt(query_result)
    print("Generated Analysis:")
    print(analysis_text)

    # Generate PDF report
    pdf_filename = "equity_analyst_report_with_analysis.pdf"
    generate_pdf_report(analysis_text, query_result, filename=pdf_filename)

    # Optionally, start interactive chat
    while True:
        user_choice = input("\nWould you like to start an interactive chat with ChatGPT? (yes/no): ").strip().lower()
        if user_choice in ["yes", "y"]:
            interactive_chat()
            break
        elif user_choice in ["no", "n"]:
            print("Exiting the program.")
            break
        else:
            print("Please enter 'yes' or 'no'.")

# -------------------------------
# 9. Entry Point
# -------------------------------

if __name__ == "__main__":
    main()
